# BasicRobotUsage  
See the project it.unibo.basicrobotusage<br/>
This is an example of an external component (an 'alien') that sends move commands 
to the basicrobot on port 8018, via TCP

## Using the basic robot via TCP

In [ ]:
##############################################################
# forward       : sends a command in output
# read          : acquires data from input
##############################################################
import socket
import time
 
robotName      = "basicrobot"  
hostAdress     = 'localhost'
basicRobotPort = 8018  

msgTemplate   = "msg(cmd,dispatch,python,"+ robotName +",cmd(CMDVAL),1)"
eventTemplate = "msg(userCmd,event,python,none,userCmd(CMDVAL),1)"
sock          = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

def connect(port) :
    server_address = (hostAdress, port)
    sock.connect(server_address)    
    print("CONNECTED WITH ", server_address)

def terminate() :
    sock.close()    #qak infrastr receives a msg null
    print("BYE")

def read() :
    BUFFER_SIZE = 1024
    data = sock.recv(BUFFER_SIZE)
    print( "received data:", data )

def forward( cmd ) :
    message = msgTemplate.replace("CMDVAL", cmd)
    print("forward ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)

def emit( cmd ) :
    message = eventTemplate.replace("CMDVAL", cmd)
    print("emit ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)
    
def console() :  
    print("console  STARTS :"   )
    cmd =  str( input() )
    print("console  cmd= :" , cmd  )
    while( cmd != "q"  ) :
        #forward( cmd )
        emit( cmd )
        #read()                  #WARNING: blocking ...
        cmd =  str(input())

##################################################
print("STARTING ... ")
connect(basicRobotPort)
sock.settimeout(60000)
console() 
terminate()

STARTING ... 
CONNECTED WITH  ('localhost', 8018)
console  STARTS :
w
console  cmd= : w
emit  msg(userCmd,event,python,none,userCmd(w),1)
h
emit  msg(userCmd,event,python,none,userCmd(h),1)


## Using the virtual robot via TCP

In [ ]:
##############################################################
# clientToWenv.py
# sendToVirtualRobot : sends a command in output
# read               : acquires data from input
##############################################################
import socket
import time
import json

sock             = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
virtualRobotPort = 8999
sep = ';'   #required by the Soffritti virtual robot 'language'

wMsg   = '{"type": "moveForward",   "arg": -1  }'
sMsg   = '{"type": "moveBackward",  "arg": -1  }'
dMsg   = '{"type": "turnLeft",      "arg": 400 }'
aMsg   = '{"type": "turnRight",     "arg": 400 }'
hMsg   = '{"type": "alarm",         "arg": 0   }'

#Python does not have a switch or case statement:use dictionary mapping
def buildMsg( move ) :    
    switcher = { 
        "w": wMsg,   
        "s": sMsg,
        "a": aMsg,
        "d": dMsg,
        "h": hMsg,         
    }
    return switcher.get(move, hMsg)

def connect(port) :
    server_address = ('localhost', port)
    sock.connect(server_address)    
    print("CONNECTED WITH the virtual robot" , server_address)

def sendToVirtualRobot( message ) :
    print("sendToVirtualRobot ", message)
    msgJson = json.loads( message )
    msg     = sep+ json.dumps(msgJson)+ sep 
    byt=msg.encode()    #required in Python3
    sock.send( byt )

def read() :
    BUFFER_SIZE = 1024
    data = sock.recv(BUFFER_SIZE)
    print( "received data:", data )

def terminate() :
    sock.close()
    print("BYE")

def console() :  
    print("console  STARTS :"   )
    cmd =  str( input() )
    print("console  cmd= :" , cmd  )   # w,s,a,d,r,l,z,x
    while( cmd != "q"  ) :
        msg = buildMsg(cmd)  
        print( msg )
        sendToVirtualRobot( msg )
        #read() #blocking ...
        cmd = str(input() )
    
###########################################    
connect( virtualRobotPort )
console()
#terminate()